## Q1

In [ ]:
import numpy as np
import cv2

def generateColorfulDogAndBallImage(width, height):
    # Create a white RGB image (3 channels)
    image = np.full((height, width, 3), 255, dtype=np.uint8)

    # Draw a Colorful Dog
    # Body - Brown
    cv2.rectangle(image, (60, 150), (130, 200), color=(42, 42, 165), thickness=-1)

    # Head - Lighter Brown
    cv2.rectangle(image, (40, 160), (60, 190), color=(60, 100, 180), thickness=-1)

    # Legs - Darker Brown
    cv2.rectangle(image, (65, 200), (70, 220), color=(20, 60, 120), thickness=-1)
    cv2.rectangle(image, (120, 200), (125, 220), color=(20, 60, 120), thickness=-1)

    # Tail - Orange
    cv2.rectangle(image, (130, 170), (140, 175), color=(0, 140, 255), thickness=-1)

    # Ear - Dark Brown
    cv2.circle(image, (45, 160), radius=5, color=(10, 40, 100), thickness=-1)

    # Draw the Ball
    cv2.circle(image, center=(220, 120), radius=15, color=(0, 0, 255), thickness=-1) 

    return image

def addGaussianNoiseColor(image):
    mean = 0
    stddev = 30
    noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
    image_float = image.astype(np.float32)
    noisy_image = np.clip(image_float + noise, 0, 255).astype(np.uint8)
    return noisy_image

# Generate the base colorful image
generatedImage = generateColorfulDogAndBallImage(600, 600)
cv2.imshow("Original Colorful Image", generatedImage)
# save the generated image
cv2.imwrite("colorful_dog_and_ball.png", generatedImage)

# Add Gaussian noise to the color image
noisyImage = addGaussianNoiseColor(generatedImage)
cv2.imshow("Noisy Color Image", noisyImage)
# save the noisy image
cv2.imwrite("noisy_colorful_dog_and_ball.png", noisyImage)

# Convert to grayscale for Otsu's thresholding
grayImage = cv2.cvtColor(noisyImage, cv2.COLOR_BGR2GRAY)
_, otsuThreshold = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow("Otsu's Thresholding", otsuThreshold)
# save the Otsu's thresholded image
cv2.imwrite("otsu_thresholded_colorful_dog_and_ball.png", otsuThreshold)

cv2.waitKey(0)
cv2.destroyAllWindows()


## Q2


In [ ]:
import cv2

# Load the image in color (do NOT convert to grayscale)
image = cv2.imread('Q2/image.jpg')

# Make a copy of the original image
image_with_seeds = image.copy()

# Define seed points (x, y)
seed_points = [(200, 100), (500, 100), (700, 100), (500, 500), (500, 300)]

# Mark seed points on the image (draw small red circles)
for (x, y) in seed_points:
    cv2.circle(image_with_seeds, (x, y), radius=5, color=(0, 0, 255), thickness=-1)  # Red color in BGR

# Display original and marked image
cv2.imshow("Original Image", image)
cv2.imshow("Image with Seed Points", image_with_seeds)
cv2.imwrite("image_with_seed_points.png", image_with_seeds)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

def show_segmentation(mask):
    cv2.imshow('Segmentation Process', mask)
    cv2.waitKey(1)  # short delay to update display

def region_growing(image, seed_points, threshold_range):
    # Initialize mask: 0 = background/unvisited, 255 = segmented/visited
    mask = np.zeros_like(image, dtype=np.uint8)
    
    queue = list(seed_points)
    
    iteration = 0
    while queue:
        iteration += 1
        current_point = queue.pop(0)
        x, y = current_point
        
        current_value = image[y, x]
        
        # Mark current point as segmented
        mask[y, x] = 255
        
        if iteration % 10 == 0:
            show_segmentation(mask)
        
        # Check 8 neighbors
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i == 0 and j == 0:
                    continue
                
                nx, ny = x + i, y + j
                
                # Check boundaries
                if 0 <= nx < image.shape[1] and 0 <= ny < image.shape[0]:
                    neighbor_value = image[ny, nx]
                    
                    # If neighbor pixel within threshold and not yet segmented
                    if abs(int(neighbor_value) - int(current_value)) <= threshold_range and mask[ny, nx] == 0:
                        queue.append((nx, ny))
                        mask[ny, nx] = 255
                        
    return mask

# Load image in grayscale
image = cv2.imread('Q2/image.jpg', cv2.IMREAD_GRAYSCALE)

# Seed points (x, y)
seed_points = [(200, 100), (500, 100), (700, 100), (500, 500), (500, 300)]

# Threshold for similarity
threshold_range = 10

# Run region growing segmentation
segmented_image = region_growing(image, seed_points, threshold_range)

cv2.destroyAllWindows()
cv2.imshow('Segmented Image', segmented_image)
cv2.imwrite('segmented_image.png', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
